In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [45]:
class extractor():
    def __init__(self):
        self.res = {}
    
    def _get_content_(self, link):
        link = "https://www.sec.gov/Archives/" + link
        webpage_response = requests.get(link,headers={'User-Agent': 'Mozilla/5.0'})
        if webpage_response.status_code == 200: 
        # The HTTP 200 OK success status response code indicates that the request has succeeded. 
            body = webpage_response.text
        else:
            print("Unable to get response with Code : %d " % (webpage_response.status_code))
        return body

    def _parse_content_(self, content, sec):
        # Regex to find <DOCUMENT> tags
        doc_start_pattern = re.compile(r'<DOCUMENT>')
        doc_end_pattern = re.compile(r'</DOCUMENT>')
        # Regex to find <TYPE> tag prceeding any characters, terminating at new line
        type_pattern = re.compile(r'<TYPE>[^\n]+')

        doc_start_is = [x.end() for x in doc_start_pattern.finditer(content)]
        doc_end_is = [x.start() for x in doc_end_pattern.finditer(content)]
        doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(content)]
        # Create a loop to go through each section type and save only the 10-K section in the dictionary
        content_n = ''
        for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
            if doc_type == '10-K':
                content_n += content[doc_start:doc_end]
        # Write the regex
        regex = re.compile(r'(>Item(\s|&#160;|&nbsp;)(1A|1B|1|2|3|4|5|7A|7|8)\.{0,1})|(ITEM(\s|&#160;|&nbsp;)(1A|1B|1|2|3|4|5|7A|7|8))')

        # Use finditer to math the regex
        matches = regex.finditer(content_n)
        # Create the dataframe
        test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])
        test_df.columns = ['item', 'start', 'end']
        test_df['item'] = test_df.item.str.lower()

        # Get rid of unnesesary charcters from the dataframe
        test_df.replace('&#160;',' ',regex=True,inplace=True)
        test_df.replace('&nbsp;',' ',regex=True,inplace=True)
        test_df.replace(' ','',regex=True,inplace=True)
        test_df.replace('\.','',regex=True,inplace=True)
        test_df.replace('>','',regex=True,inplace=True)


        # Drop duplicates
        count1a = len(test_df.loc[test_df['item']=='item1a'])
        if count1a > 1:
            pos_dat = test_df.sort_values('start', ascending=True)[10:].drop_duplicates(subset=['item'], keep='first')
        else:
            pos_dat = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='first')

        # Set item as the dataframe index
        pos_dat.set_index('item', inplace=True)

        firm_dict = {}
        item_lst = ['item1','item1a','item1b','item2','item3','item4','item5','item7','item7a','item8']
        for i in sec:
            if i == 'item8' or i not in item_lst:
                print("Item out of index")
                continue
            sec_content = content_n[pos_dat['start'].loc[i]:pos_dat['start'].loc[item_lst[item_lst.index(i)+1]]]
            firm_dict[i] = self._prettify_(sec_content)
        return firm_dict

    def _prettify_(self,text):
        parsed_content = BeautifulSoup(text, 'lxml')
        text_part = parsed_content.get_text("\n\n")
        return text_part

    def extract(self, cik_lst,link_lst, sec_lst):     # sec:item1a, item1, item1b
        for i in range(len(link_lst)):
            self.res[cik_lst[i]] = self._parse_content_(self._get_content_(link_lst[i]),sec_lst)
        return self.res

In [3]:
'''
exampleset = [('0000001800',
  'ABBOTT LABORATORIES',
  '10-K',
  'edgar/data/1800/0001047469-19-000624.txt'),
 ('0000796343',
  'ADOBE INC.',
  '10-K',
  'edgar/data/796343/0000796343-19-000019.txt'),
 ('0000002488',
  'ADVANCED MICRO DEVICES INC',
  '10-K',
  'edgar/data/2488/0000002488-19-000011.txt'),
 ('0000764180',
  'ALTRIA GROUP, INC.',
  '10-K',
  'edgar/data/764180/0000764180-19-000023.txt')]'''

In [ ]:
'''
# test case
cik_lst = [i[0] for i in exampleset]
link_lst = [i[3] for i in exampleset]
sec_lst = ('item1','item1a') # attention: section list should be in tuples

extr = extractor()
test_res = extr.extract(cik_lst,link_lst,sec_lst)
'''